# CASSANDRA

##### Crear la BBDD

CREATE KEYSPACE incidents
 WITH replication = {
'class':'SimpleStrategy',
 'replication_factor':1
}

##### Crear consulta principal

En este punto en el que nos encontramos de desarrollo, para el prototipo vamos a optar por tener una única consulta en Cassandra, que contendrá todos los datos del CSV original.
La idea es, a partir de aquí, crear subtablas que contengan categorías y número de incidencias por cada una por poner un ejemplo. 
Es decir, consultas para agilizar el proceso y tener acceso a la información de manera mucho más inmediata que realizando búsquedas sobre esta primera consulta original como vamos a hacer a continuación.
Junto a ello, también se incluirán gráficos para ver de manera mucho mas simplificada y directa la información que vamos a representar ahora en tablas.

##### Tabla principal
CREATE TABLE incidents.datosbasicos(
	incidentNumber text,
	category text,
	descript text,
	dayofweek text,
	date date,
	time time,
	pddistrict text,
	resolution text,
	address text,
	x text,
	y text,
	location text,
	pdid text,
	PRIMARY KEY(incidentNumber, category,dayofweek, date, time, pddistrict, resolution, address, x, y, location, pdid)
);

##### Inserción de datos
Tras la creación, el siguiente paso es introducir los datos de que disponemos a través del CSV en esta primera consulta.

COPY incidents.datosbasicos (incidentNumber, category, descript, dayofweek, date, time, pddistrict, resolution, address, x, y, location, pdid) FROM 'C:\...\CSV.csv' WITH HEADER=TRUE and CHUNKSIZE=500 and INGESTRATE=2000;

##### Exportación de datos
Si por algún motivo queremos exportar alguna consulta a un CSV el proceso es el siguiente:

COPY incidents.datosbasicos (incidentNumber, category, descript, dayofweek, date, time, pddistrict, resolution, address, x, y, location, pdid) TO 'C:\...\temp.csv';

##### Borrar consulta
Por último, si queremos borrar una consulta no tenemos más que escribir:
DROP TABLE keyspace.table;

### Consultas realizadas
Hemos desarrollado las siguientes consultas en nuestro prototipo de Python con Cassandra:
    - Listar todos los datos disponibles en el CSV
    - Ver las cinco categorías con más incidentes
    - Ver el número de incidencias por día de la semana
    - Ver los cinco distritos con mayor número de incidencias
    - Buscar el número de incidentes aquellos que contengan cierta palabra en la descripción ('DOG' en nuestro ejemplo)
    - Ver el número de incidencias por bloques de días, esto es, del día 1 al 10, del 11 al 20, y del 21 hasta el resto.


In [1]:
from cassandra.cluster import Cluster
from cassandra.query import dict_factory
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

cluster = Cluster()
session = cluster.connect('incidents')
session.row_factory = dict_factory

In [2]:
queryDatosBasicos = "SELECT * FROM datosbasicos"
dfDatosBasicos = pd.DataFrame(list(session.execute(queryDatosBasicos)))
dfDatosBasicos

,address,category,date,dayofweek,descript,incidentnumber,location,pddistrict,pdid,resolution,time,x,y
0,100 Block of 3RD ST,PROSTITUTION,2016-01-29,Friday,SOLICITS FOR ACT OF PROSTITUTION,160084328,"(37.78545749029967, -122.40091602902488)",SOUTHERN,16008432813060,"ARREST, BOOKED",14:00:00.000000000,-122.40091602902487,37.78545749029967
1,100 Block of 3RD ST,PROSTITUTION,2016-01-29,Friday,LOITERING FOR PURPOSE OF PROSTITUTION,160084328,"(37.78545749029967, -122.40091602902488)",SOUTHERN,16008432813075,"ARREST, BOOKED",14:00:00.000000000,-122.40091602902487,37.78545749029967
2,PINE ST / STOCKTON ST,LARCENY/THEFT,2006-06-03,Saturday,GRAND THEFT FROM LOCKED AUTO,60589373,"(37.7912964515689, -122.40734222874)",CENTRAL,6058937306244,NONE,18:00:00.000000000,-122.40734222873999,37.7912964515689
3,0 Block of TUCKER AV,VANDALISM,2008-11-30,Sunday,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",81280257,"(37.7155319865115, -122.404500268176)",INGLESIDE,8128025728160,NONE,01:00:00.000000000,-122.40450026817601,37.7155319865115
4,TURK ST / TAYLOR ST,DRUG/NARCOTIC,2008-07-30,Wednesday,POSSESSION OF BASE/ROCK COCAINE FOR SALE,80799853,"(37.7832145190013, -122.410768766343)",TENDERLOIN,8079985316623,"ARREST, BOOKED",06:42:00.000000000,-122.410768766343,37.7832145190013
5,1100 Block of SCOTT ST,VANDALISM,2005-11-10,Thursday,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",51269758,"(37.7803348745141, -122.437236101904)",PARK,5126975828160,NONE,01:20:00.000000000,-122.43723610190399,37.780334874514104
6,0 Block of DANIELBURNHAM CT,LARCENY/THEFT,2007-02-01,Thursday,PETTY THEFT FROM LOCKED AUTO,70114794,"(37.7870192301761, -122.42254731779)",NORTHERN,7011479406242,NONE,08:20:00.000000000,-122.42254731778999,37.7870192301761
7,0 Block of TINGLEY ST,NON-CRIMINAL,2008-01-19,Saturday,LOST PROPERTY,86006127,"(37.7287340517921, -122.432948738078)",INGLESIDE,8600612771000,NONE,10:00:00.000000000,-122.432948738078,37.7287340517921
8,1300 Block of FELL ST,BURGLARY,2005-02-06,Sunday,"BURGLARY OF APARTMENT HOUSE, UNLAWFUL ENTRY",50147088,"(37.7737933965852, -122.440126155736)",PARK,5014708805013,NONE,16:00:00.000000000,-122.44012615573601,37.773793396585205
9,400 Block of BROADWAY ST,ASSAULT,2013-01-29,Tuesday,THREATS AGAINST LIFE,130082073,"(37.7982249345487, -122.404282497745)",CENTRAL,13008207319057,NONE,13:00:00.000000000,-122.404282497745,37.7982249345487


In [3]:
categoriasMasFrecuentes = pd.DataFrame()
categoriasMasFrecuentes['Most frequent categories'] = dfDatosBasicos['category'].value_counts()
categoriasMasFrecuentes

,Most frequent categories
LARCENY/THEFT,471563
OTHER OFFENSES,305733
NON-CRIMINAL,234803
ASSAULT,191688
VEHICLE THEFT,125641
DRUG/NARCOTIC,118655
VANDALISM,114220
WARRANTS,100279
BURGLARY,90136
SUSPICIOUS OCC,79342


In [4]:
categoriasMasFrecuentes.to_csv('categoriasMasFrecuentes.csv')

In [5]:
diasSemanaMasFrecuentes = pd.DataFrame()
diasSemanaMasFrecuentes['Most frequent days'] = dfDatosBasicos['dayofweek'].value_counts()
diasSemanaMasFrecuentes

,Most frequent days
Friday,333109
Wednesday,319498
Saturday,316019
Thursday,311705
Tuesday,310977
Monday,302397
Sunday,290528


In [6]:
diasSemanaMasFrecuentes.to_csv('diasSemanasMasFrecuentes.csv')

In [7]:
distritosMasFrecuentes = pd.DataFrame()
distritosMasFrecuentes['Most frequent districts'] = dfDatosBasicos['pddistrict'].value_counts()
distritosMasFrecuentes

,Most frequent districts
SOUTHERN,393983
MISSION,295709
NORTHERN,268788
CENTRAL,222217
BAYVIEW,218359
INGLESIDE,191727
TENDERLOIN,189859
TARAVAL,164600
PARK,123799
RICHMOND,115192


In [8]:
distritosMasFrecuentes.to_csv('distritosMasFrecuentes.csv')

In [9]:
descripciones = pd.DataFrame()
descripciones['Descriptions'] = dfDatosBasicos['descript']
descripciones


,Descriptions
0,SOLICITS FOR ACT OF PROSTITUTION
1,LOITERING FOR PURPOSE OF PROSTITUTION
2,GRAND THEFT FROM LOCKED AUTO
3,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES"
4,POSSESSION OF BASE/ROCK COCAINE FOR SALE
5,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES"
6,PETTY THEFT FROM LOCKED AUTO
7,LOST PROPERTY
8,"BURGLARY OF APARTMENT HOUSE, UNLAWFUL ENTRY"
9,THREATS AGAINST LIFE


In [10]:
ocurrenciasDOG = pd.DataFrame()
ocurrenciasDOG = dfDatosBasicos['descript'][dfDatosBasicos['descript'].str.contains('DOG')]
ocurrenciasDOG.count()

3677

In [11]:
diasPrimerTercio = pd.Series()
diasSegundoTercio = pd.Series()
diasTercerTercio = pd.Series()
diasFuenteOriginal = pd.DataFrame()

In [12]:
def sacarNumDia(date):
    date_aux = str(date)
    return str(date_aux[8:10])

In [13]:
diasFuenteOriginal = dfDatosBasicos['date'].apply(sacarNumDia)

In [14]:
dataAux1 = []
dataAux2 = []
dataAux3 = []

In [15]:
for day in diasFuenteOriginal:
    aux = int(day)
    if(aux<=10):
        dataAux1.append(day)
    elif(aux>10 and aux<=20):
        dataAux2.append(day)
    else:
        dataAux3.append(day)

In [16]:
diasPrimerTercio = pd.DataFrame(dataAux1)
diasPrimerTercio.count()

0    724005
dtype: int64

In [17]:
diasSegundoTercio = pd.DataFrame(dataAux2)
diasSegundoTercio.count()

0    718393
dtype: int64

In [18]:
diasTercerTercio = pd.DataFrame(dataAux3)
diasTercerTercio.count()

0    741835
dtype: int64